In [151]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [152]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [153]:
 baseurl = 'https://vast-depths-89393.herokuapp.com/'
# baseurl = 'http://127.0.0.1:5000/'

In [154]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [155]:
response.json()

{'signingKey_hex': 'df35903ff0311694685bafcf832f83f45d0e7c1f985918ef9e67397d9d77ab79',
 'traderId': '39',
 'verifyKey_hex': '099ba938eafaf19ae20db8357b917322d248faf3b11182e742832ec3847f2990'}

In [156]:
url = baseurl + 'createMarket'

mkId = 6
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [157]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [158]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,6,39


In [159]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.5678, "quantity":1}

In [160]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [ ]:
# View json response
pd.DataFrame(response.json(), index=[0])

In [163]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [ ]:
response.json()

In [165]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 62.04933977127075 seconds.
matched trade: 1 in 59.610429763793945 seconds.
matched trade: 2 in 58.92987322807312 seconds.
matched trade: 3 in 53.34968185424805 seconds.
matched trade: 4 in 55.25232815742493 seconds.
matched trade: 5 in 54.6832320690155 seconds.
matched trade: 6 in 51.3498330116272 seconds.
matched trade: 7 in 50.68759870529175 seconds.
matched trade: 8 in 58.270055055618286 seconds.
matched trade: 9 in 53.322766065597534 seconds.


In [114]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [166]:
response.json()

{'checks': 'ProbablyASignatureError',
 'marketBranchId': 1,
 'marketRootId': 6,
 'price': 0.5678,
 'quantity': 1,
 'traderId': '39'}

In [167]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [117]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,1.0
1,2,1,0,0.7
2,3,1,0,0.7
3,4,1,0,0.7
4,6,1,0,1.0


In [168]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [169]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.5510,1,1
1,1,1,0.5510,-1,1
10,1,1,0.5510,1,1
100,1,1,0.5510,-1,4
101,1,1,0.5510,1,4
102,1,1,0.5510,1,4
103,1,1,0.5510,-1,4
104,1,1,0.5510,-1,4
105,1,1,0.5510,1,4
106,1,1,0.5510,-1,4


In [170]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 24.23835015296936 seconds.
matched trade: 1 in 18.970323085784912 seconds.
matched trade: 2 in 19.54018783569336 seconds.
matched trade: 3 in 20.687762022018433 seconds.
matched trade: 4 in 18.369783639907837 seconds.
matched trade: 5 in 18.592928886413574 seconds.
matched trade: 6 in 29.398239850997925 seconds.
matched trade: 7 in 31.328058004379272 seconds.
matched trade: 8 in 30.595832109451294 seconds.
matched trade: 9 in 19.711176872253418 seconds.


In [120]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [121]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.440,10,3
1,1,1,0.440,10,4
2,1,1,0.440,10,5
3,1,1,0.440,10,6
4,1,1,0.551,-1,6


In [122]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [123]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.5510,0,3
1,1,1,0.5510,0,4
2,1,1,0.5510,-1,5
3,1,1,0.5510,1,6
4,6,1,0.5678,0,8


In [99]:
tId

'6'